<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/FSML_Python/chap10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第10章 アンサンブル学習

アンサンブル学習で、breast\_cancerデータの識別を行います。breast\_cancerデータは腫瘍の画像から取り出した半径や周囲の値の平均・標準偏差・最大値などを特徴とし、その腫瘍が悪性(malignant)か良性(benign)かという正解が付いたデータです。

ライブラリの読み込み

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.datasets import load_breast_cancer
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn import tree

breast_cancerデータを読み込んでX, yに格納後、概要を表示します。

In [ ]:
bc = load_breast_cancer(as_frame=True)
X = bc.data
y = bc.target
print(bc.DESCR)

In [ ]:
X

In [ ]:
y

可視化によって問題の難しさの検討をつけます。一部に重なりはありますが、比較的識別しやすいデータのように見えます。

In [ ]:
tsne = TSNE(n_components=2, perplexity=5, random_state=1)
X2 = tsne.fit_transform(X)
plt.plot(X2[y==0,0], X2[y==0,1], 'rx', label=bc.target_names[0])
plt.plot(X2[y==1,0], X2[y==1,1], 'bo', label=bc.target_names[1])
plt.legend()
plt.show()

## バギング

base_estimator引数で識別器を指定します。指定しない場合は決定木になります。

In [ ]:
clf1 = ensemble.BaggingClassifier()

交差確認法で評価します。

In [ ]:
scores = cross_val_score(clf1, X, y, cv=10)
print(f'{scores.mean() * 100:4.2f} +/- {scores.std() * 200:4.2f} %')

弱識別器として作成された決定木の根に近い部分を表示します。比較的似ている決定木になっていることがわかります。

In [ ]:
clf1.fit(X, y)

fig, ax = plt.subplots(len(clf1.estimators_),1, figsize=(10,30))

for i, t in enumerate(clf1.estimators_):
  tree.plot_tree(t, ax=ax[i], fontsize=10, max_depth=2)

## ランダムフォレスト

In [ ]:
clf2 = ensemble.RandomForestClassifier(n_estimators=10)

交差確認法で評価します。

In [ ]:
scores = cross_val_score(clf2, X, y, cv=10)
print(f'{scores.mean() * 100:4.2f} +/- {scores.std() * 200:4.2f} %')

弱識別器として作成された決定木の根に近い部分を表示します。バギングと比べるとある程度異なった決定木になっていることがわかります。

In [ ]:
clf2.fit(X, y)

fig, ax = plt.subplots(len(clf2.estimators_),1, figsize=(10,30))

for i, t in enumerate(clf2.estimators_):
  tree.plot_tree(t, ax=ax[i], fontsize=10, max_depth=2)

## Adaブースト

In [ ]:
clf3 = ensemble.AdaBoostClassifier()

交差確認法で評価します。

In [ ]:
scores = cross_val_score(clf3, X, y, cv=10)
print(f'{scores.mean() * 100:4.2f} +/- {scores.std() * 200:4.2f} %')

Adaブーストの弱識別器のデフォルトは深さ1の決定木です。先頭の10個を確認すると、ほとんど異なる特徴が選ばれています。

In [ ]:
clf3.fit(X, y)

fig, ax = plt.subplots(10 ,1, figsize=(10,30))

for i, t in zip(range(10), clf3.estimators_[:10]):
  tree.plot_tree(t, ax=ax[i], fontsize=10)

## 勾配ブースティング

In [ ]:
clf4 = ensemble.GradientBoostingClassifier()

交差確認法で評価します。

In [ ]:
scores = cross_val_score(clf4, X, y, cv=10)
print(f'{scores.mean() * 100:4.2f} +/- {scores.std() * 200:4.2f} %')

各特徴の重要性を確認することができます。ここでは上位10個まで表示します。

In [ ]:
clf4.fit(X,y)
importances = clf4.feature_importances_
indices = np.argsort(importances)[::-1]

plt.title("Feature importances")
plt.barh(bc.feature_names[indices[10::-1]], importances[indices[10::-1]])
plt.show()

## カテゴリ特徴を含む場合

次にカテゴリ特徴を含んだcredit-gデータの識別を行います。credit-gデータの解説は、教科書p.263を参照してください。

識別器は、カテゴリ特徴をそのまま入力できる
[HistGradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html)を使います。これは、二分木で分岐する値を決めるときにヒストグラムを使うことで数値もカテゴリも同様に扱うことができる方法です。

credit-gデータをpandasのDataFrame形式で読み込んでX, yに格納後、概要を表示します。

In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

ds = fetch_openml('credit-g', version = 'active', as_frame=True)
X = ds.data
y = ds.target
print(ds.DESCR)

Xの中身を確認します。

In [ ]:
X

In [ ]:
X['housing']

In [ ]:
ca = []
for t in X.dtypes:
  if t == 'category':
    ca.append(True)
  else:
    ca.append(False)
ca

In [ ]:
ca[14]=False

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

ordinal_encoder = make_column_transformer(
    (
        OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
        make_column_selector(dtype_include="category"),
    ),
    remainder="passthrough",
)

clf5 = make_pipeline(
    ordinal_encoder,
    ensemble.HistGradientBoostingClassifier(categorical_features=ca, max_bins=255)
)

In [ ]:
clf5.fit(X,y)

In [ ]:
scores = cross_val_score(clf5, X, y, cv=10)
print(f'{scores.mean() * 100:4.2f} +/- {scores.std() * 200:4.2f} %')

## 練習問題

アンサンブル学習で California housing データの回帰を行い、各特徴の重要性を確認せよ。

ライブラリの読み込み

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

California housing データを読み込んでX, yに格納後、概要を表示する。

In [ ]:
housing = fetch_california_housing()
X = housing.data
y = housing.target
print(housing.DESCR)

特徴のスケールがかなり異なるので、標準化しておく。

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

ここまでが前置き。こここからが課題。

In [ ]:
# 手法を決めて

In [ ]:
# 交差検証でスコアを出し

In [ ]:
# 各特徴の重要性を表示する
